<a href="https://colab.research.google.com/github/SathyaSudha-96/Deep-Learning-2021-22/blob/main/IDL_Assignment_6_SathyaSudhaMurugan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras import layers
import time
from keras.utils.vis_utils import plot_model

In [2]:
# remove infrequent words. you can play with this parameter as it will likely impact model quality
num_words = 20000
(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=num_words)

In [3]:
# now we can create a dataset!
#train_data = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels.astype(np.int32)))

Oops..This is not possible as tensor_slices cant make dataset out of non-rectangualr tensors.

💡Here comes **from_generator** for our rescue....

In [4]:
#First we must create a Generator
def generator():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label

In [5]:
train_data_gen = tf.data.Dataset.from_generator(
    generator,
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.int32)))


In [6]:
def test_generator():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

test_data_gen = tf.data.Dataset.from_generator(
    test_generator,
    output_signature=(
        tf.TensorSpec(shape=(None,), dtype=tf.int32),
        tf.TensorSpec(shape=(), dtype=tf.int32)))

test_data_padded = test_data_gen.padded_batch(32, drop_remainder= True)

Regular Batch wouldn't work due to variable length sequences...

> **padded_batch** can be used to batch sequences of variable length. It can batch and then pad all the sequences to longest sequence length of that batch.




In [7]:
train_data_padded = train_data_gen.padded_batch(32, drop_remainder= True)

In [8]:
for sequence, label in train_data_padded:
    print(sequence.shape, label.shape)
    #input()

(32, 888) (32,)
(32, 861) (32,)
(32, 787) (32,)
(32, 1038) (32,)
(32, 563) (32,)
(32, 810) (32,)
(32, 584) (32,)
(32, 932) (32,)
(32, 496) (32,)
(32, 839) (32,)
(32, 828) (32,)
(32, 1011) (32,)
(32, 1007) (32,)
(32, 918) (32,)
(32, 785) (32,)
(32, 985) (32,)
(32, 722) (32,)
(32, 633) (32,)
(32, 942) (32,)
(32, 1009) (32,)
(32, 939) (32,)
(32, 843) (32,)
(32, 523) (32,)
(32, 440) (32,)
(32, 646) (32,)
(32, 900) (32,)
(32, 978) (32,)
(32, 626) (32,)
(32, 935) (32,)
(32, 1000) (32,)
(32, 533) (32,)
(32, 1022) (32,)
(32, 741) (32,)
(32, 367) (32,)
(32, 985) (32,)
(32, 932) (32,)
(32, 656) (32,)
(32, 959) (32,)
(32, 1009) (32,)
(32, 502) (32,)
(32, 919) (32,)
(32, 622) (32,)
(32, 811) (32,)
(32, 872) (32,)
(32, 988) (32,)
(32, 727) (32,)
(32, 950) (32,)
(32, 736) (32,)
(32, 445) (32,)
(32, 679) (32,)
(32, 430) (32,)
(32, 723) (32,)
(32, 699) (32,)
(32, 909) (32,)
(32, 962) (32,)
(32, 664) (32,)
(32, 1015) (32,)
(32, 627) (32,)
(32, 872) (32,)
(32, 838) (32,)
(32, 854) (32,)
(32, 624) (32,)


However, batches out of padded_batch isn't efficient. For instance, if there are many short sequences along with very few long sequences in a batch. All the short sequences are padeed to the longest sequence which are were very few in the batch.


> As an alternative, we can use **Bucketing**.



In [9]:
#Lets the maximum length
sequence_lengths = [len(sequence) for sequence in train_sequences]
max_len = max(sequence_lengths)
min_len = min(sequence_lengths)
average = sum(sequence_lengths) / len(sequence_lengths)
print("Longest Sequence Length:", max_len)
print("Smallest Sequence Length:", min_len)
print("Average Sequence Length:", int(average))

Longest Sequence Length: 2494
Smallest Sequence Length: 11
Average Sequence Length: 238


In [10]:
buckets = [100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,2000,2200,2400]
bucket_batch_size = [32] * (len(buckets) + 1)
train_data_bucket = train_data_gen.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                  bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size, drop_remainder=True)

In [11]:
test_data_bucket = test_data_gen.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                  bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size, drop_remainder=True)

In [12]:
for sequence, label in train_data_bucket:
    print(sequence.shape, label.shape)

(32, 193) (32,)
(32, 198) (32,)
(32, 287) (32,)
(32, 198) (32,)
(32, 199) (32,)
(32, 296) (32,)
(32, 99) (32,)
(32, 195) (32,)
(32, 393) (32,)
(32, 199) (32,)
(32, 297) (32,)
(32, 195) (32,)
(32, 97) (32,)
(32, 496) (32,)
(32, 197) (32,)
(32, 599) (32,)
(32, 199) (32,)
(32, 297) (32,)
(32, 193) (32,)
(32, 387) (32,)
(32, 97) (32,)
(32, 196) (32,)
(32, 295) (32,)
(32, 198) (32,)
(32, 189) (32,)
(32, 295) (32,)
(32, 197) (32,)
(32, 99) (32,)
(32, 187) (32,)
(32, 397) (32,)
(32, 499) (32,)
(32, 298) (32,)
(32, 196) (32,)
(32, 694) (32,)
(32, 198) (32,)
(32, 199) (32,)
(32, 297) (32,)
(32, 199) (32,)
(32, 98) (32,)
(32, 197) (32,)
(32, 298) (32,)
(32, 198) (32,)
(32, 395) (32,)
(32, 198) (32,)
(32, 295) (32,)
(32, 198) (32,)
(32, 99) (32,)
(32, 190) (32,)
(32, 297) (32,)
(32, 186) (32,)
(32, 496) (32,)
(32, 382) (32,)
(32, 190) (32,)
(32, 196) (32,)
(32, 293) (32,)
(32, 198) (32,)
(32, 798) (32,)
(32, 96) (32,)
(32, 599) (32,)
(32, 196) (32,)
(32, 197) (32,)
(32, 397) (32,)
(32, 299) (32,)

In [18]:
LSTM_model = tf.keras.Sequential([tf.keras.layers.Embedding(num_words, 20, mask_zero=True), #mask is zero to prevent computation on padded time step
                             tf.keras.layers.LSTM(128, return_sequences=True),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dense(1)])


> For padded_batch data



In [14]:
epochs = 3
optimizer = tf.optimizers.Adam()
loss_function = tf.keras.losses.BinaryCrossentropy()
accuracy = tf.keras.metrics.BinaryAccuracy()
for epoch in range(epochs):
  print('Starting Epoch',epoch+1)
  for step, (sequence_batch, label_batch) in enumerate(train_data_padded):
    seq, lbl = sequence_batch, label_batch
    with tf.GradientTape(persistent=True) as tape:
      output = LSTM_model(seq)
      loss = loss_function(lbl,output)
    #Compute gradients  
    gradients = tape.gradient(loss,LSTM_model.trainable_variables)  
    #Apply Gradients
    optimizer.apply_gradients(zip(gradients, LSTM_model.trainable_variables))
    accuracy(lbl,output)   
    if not step % 100:
      print("Loss: {} Training Accuracy: {}".format(loss, accuracy.result()))

Starting Epoch 1
Loss: 3.6870017051696777 Training Accuracy: 0.59375
Loss: 0.6316385269165039 Training Accuracy: 0.5572401285171509
Loss: 0.6727356910705566 Training Accuracy: 0.634639322757721
Loss: 6.708292007446289 Training Accuracy: 0.6314368844032288
Loss: 6.708292007446289 Training Accuracy: 0.600062370300293
Loss: 1.0928009748458862 Training Accuracy: 0.6036052703857422
Loss: 0.5892713069915771 Training Accuracy: 0.618916392326355
Loss: 0.4727975130081177 Training Accuracy: 0.6313302516937256
Starting Epoch 2
Loss: 0.5570995807647705 Training Accuracy: 0.6375479698181152
Loss: 2.12031888961792 Training Accuracy: 0.6572420597076416
Loss: 3.967468023300171 Training Accuracy: 0.6776667237281799
Loss: 0.6581907868385315 Training Accuracy: 0.6842652559280396
Loss: 0.44566452503204346 Training Accuracy: 0.6895357370376587
Loss: 0.5050106048583984 Training Accuracy: 0.6980060338973999
Loss: 0.9906338453292847 Training Accuracy: 0.7072630524635315
Loss: 0.4748328626155853 Training Accur

In [17]:
test_accuracy = tf.keras.metrics.BinaryAccuracy()
for step, (sequence_batch, label_batch) in enumerate(test_data_padded):
  seq, lbl = sequence_batch, label_batch
  output = LSTM_model(seq)
  loss = loss_function(lbl,output)
  test_accuracy(lbl,output)
print('Testing Accuracy:', format(test_accuracy.result())) 

Testing Accuracy: 0.537291944026947




---





> For Bucket Data



In [14]:
epochs = 3
optimizer = tf.optimizers.Adam()
loss_function = tf.keras.losses.BinaryCrossentropy()
accuracy = tf.keras.metrics.BinaryAccuracy()
for epoch in range(epochs):
  print('Starting Epoch',epoch+1)
  for step, (sequence_batch, label_batch) in enumerate(train_data_bucket):
    seq, lbl = sequence_batch, label_batch
    with tf.GradientTape(persistent=True) as tape:
      output = LSTM_model(seq)
      loss = loss_function(lbl,output)
    #Compute gradients  
    gradients = tape.gradient(loss,LSTM_model.trainable_variables)  
    #Apply Gradients
    optimizer.apply_gradients(zip(gradients, LSTM_model.trainable_variables))
    accuracy(lbl,output)   
    if not step % 100:
      print("Loss: {} Training Accuracy: {}".format(loss, accuracy.result()))

Starting Epoch 1
Loss: 4.24458122253418 Training Accuracy: 0.6875
Loss: 0.6224971413612366 Training Accuracy: 0.5714727640151978
Loss: 0.4199749231338501 Training Accuracy: 0.6386815905570984
Loss: 0.5049973726272583 Training Accuracy: 0.6662167906761169
Loss: 0.334351509809494 Training Accuracy: 0.6935785412788391
Loss: 0.42034614086151123 Training Accuracy: 0.7074601054191589
Loss: 0.32707202434539795 Training Accuracy: 0.7183340191841125
Loss: 0.5383660197257996 Training Accuracy: 0.7244115471839905
Starting Epoch 2
Loss: 0.34062421321868896 Training Accuracy: 0.7270350456237793
Loss: 0.3108678460121155 Training Accuracy: 0.7353192567825317
Loss: 0.3994215130805969 Training Accuracy: 0.7422594428062439
Loss: 0.288842111825943 Training Accuracy: 0.7485782504081726
Loss: 0.5919021368026733 Training Accuracy: 0.7526816129684448
Loss: 0.3890606760978699 Training Accuracy: 0.7540132999420166
Loss: 0.34497854113578796 Training Accuracy: 0.7571486830711365
Loss: 0.8042756915092468 Training

In [15]:
test_accuracy = tf.keras.metrics.BinaryAccuracy()
for step, (sequence_batch, label_batch) in enumerate(test_data_bucket):
  seq, lbl = sequence_batch, label_batch
  output = LSTM_model(seq)
  loss = loss_function(lbl,output)
  test_accuracy(lbl,output)
print('Testing Accuracy:', format(test_accuracy.result())) 

Testing Accuracy: 0.8207257986068726




---



GRU Model

In [15]:
GRU_model = tf.keras.Sequential([tf.keras.layers.Embedding(num_words, 20, mask_zero=True), 
                             tf.keras.layers.GRU(128, return_sequences=True),
                             tf.keras.layers.GRU(64),
                             tf.keras.layers.Dense(1)])
optimizer = tf.optimizers.Adam()
loss_fn = tf.losses.BinaryCrossentropy()
accuracy_metric = tf.metrics.BinaryAccuracy()
GRU_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])
GRU_model.fit(train_data_bucket, epochs=3, validation_data=test_data_bucket)

Epoch 1/3
776/776 [==============================] - 140s 169ms/step - loss: 0.6155 - binary_accuracy: 0.6750 - val_loss: 0.4794 - val_binary_accuracy: 0.7864
Epoch 2/3
776/776 [==============================] - 125s 161ms/step - loss: 0.5161 - binary_accuracy: 0.8141 - val_loss: 0.4953 - val_binary_accuracy: 0.8057
Epoch 3/3
776/776 [==============================] - 123s 159ms/step - loss: 0.3143 - binary_accuracy: 0.9041 - val_loss: 0.6599 - val_binary_accuracy: 0.8610




---





---



Bidirectional RNN

In [16]:
LSTM_BiDirectional_model = tf.keras.Sequential([tf.keras.layers.Embedding(num_words, 20, mask_zero=True), #mask is zero to prevent computation on padded time step
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(1)])
optimizer = tf.optimizers.Adam()
loss_fn = tf.losses.BinaryCrossentropy()
accuracy_metric = tf.metrics.BinaryAccuracy()
LSTM_BiDirectional_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])
LSTM_BiDirectional_model.fit(train_data_bucket, epochs=3, validation_data=test_data_bucket)


Epoch 1/3
776/776 [==============================] - 242s 290ms/step - loss: 0.5807 - binary_accuracy: 0.7531 - val_loss: 3.5614 - val_binary_accuracy: 0.5083
Epoch 2/3
776/776 [==============================] - 217s 279ms/step - loss: 3.7387 - binary_accuracy: 0.7544 - val_loss: 4.4953 - val_binary_accuracy: 0.7063
Epoch 3/3
776/776 [==============================] - 215s 277ms/step - loss: 3.9228 - binary_accuracy: 0.7436 - val_loss: 4.4953 - val_binary_accuracy: 0.7063


In [14]:
LSTM_BiDirectional_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 20)          400000    
                                                                 
 bidirectional (Bidirectiona  (None, None, 24)         3168      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 30)               4800      
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 31        
                                                                 
Total params: 407,999
Trainable params: 407,999
Non-trainable params: 0
_________________________________________________________________




---



Ragged Tesnsor

In [15]:
ragged_Train_sequence = tf.ragged.constant(train_sequences)
ragged_Test_sequence = tf.ragged.constant(test_sequences)

In [17]:
#now we can create a dataset!
ragged_train_data = tf.data.Dataset.from_tensor_slices((ragged_Train_sequence, train_labels.astype(np.int32)))
ragged_test_data = tf.data.Dataset.from_tensor_slices((ragged_Test_sequence, train_labels.astype(np.int32)))

padding is not required as tensorflow operation supports Ragged Tensor.

In [48]:
ragged_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[None], dtype=tf.int32, ragged=True),
    tf.keras.layers.Embedding(num_words,128),
    tf.keras.layers.LSTM(32, use_bias=False),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Activation(tf.nn.relu),
    tf.keras.layers.Dense(1)
])

In [55]:
optimizer = tf.optimizers.Adam()
loss_fn = tf.losses.BinaryCrossentropy()
accuracy_metric = tf.metrics.BinaryAccuracy()
ragged_model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])
ragged_model.fit(ragged_Train_sequence,train_labels, epochs=1, batch_size=32, validation_data=(ragged_Test_sequence,test_labels))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_6/lstm_16/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_6/lstm_16/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_6/lstm_16/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


782/782 [==============================] - 1885s 2s/step - loss: 0.7109 - binary_accuracy: 0.6965 - val_loss: 0.8327 - val_binary_accuracy: 0.5058
